<a href="https://colab.research.google.com/github/Shourya306/Unsupervised-Machine-Learning/blob/main/Online_Retail_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Online Retail Customer Segmentation. </u></b>

## <b> Problem Description </b>

### In this project, your task is to identify major customer segments on a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## <b> Data Description </b>

### <b>Attribute Information: </b>

* ### InvoiceNo: Invoice number. A 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* ### StockCode: Product (item) code. A 5-digit integral number uniquely assigned to each distinct product.
* ### Description: Product (item) name.
* ### Quantity: The quantities of each product (item) per transaction.
* ### InvoiceDate: Invoice Date and time. The day and time when each transaction was generated.
* ### UnitPrice: Unit price. Product price per unit in sterling.
* ### CustomerID: Customer number. A 5-digit integral number uniquely assigned to each customer.
* ### Country: Country name. The name of the country where each customer resides.

In [25]:
# Importing the pre-requisite libraries.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()
import datetime as dt
import math

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')   # Code to ensure that python dosesn't give any warnings.

pd.set_option('max_columns', None) # Code to ensure that all the columns in the dataframe are shown.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
# Importing the dataset.

retail_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Online Retail Customer Segmentation - Challa Shourya Chandra Sai/Online Retail.xlsx')

In [27]:
# Inspecting the dataset.

retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [28]:
retail_df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [29]:
# Understanding the basic stats.

retail_df.describe(include = 'all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909.0,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900.0,4070,4223,NaN,23260,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom
freq,1114.0,2313,2369,NaN,1114,NaN,NaN,495478
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,9.552250,NaN,4.611114,15287.690570,NaN
std,NaN,NaN,NaN,218.081158,NaN,96.759853,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,13953.000000,NaN


In [30]:
# Checking out the datatype of all the columns.

retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


## **Defining The Problem Statement.**

*Understanding the difference between numeric[continuous variables and discrete variables] & categorical variables.*

**Continous variables** - These are those variables whose values are obtained by measuring. They can assume any value between an interval.

**Discrete variables** - These are those variables whose values are obtained by counting. They can assume distinct set of values.

**Categorical variables** - These are those variables whose values are of string datatype or binary in nature and they assume distinct set of values.


*Segregating the variables into categorical, continuous/discrite, datetime and, textual in nature.*

Variables:

* InvoiceNo = It is categorical and Nominal in nature.

* StockCode = It is categorical and Nominal in nature.

* Description = It is textual in nature. 

* Quantity = It is continuous in nature.

* InvoiceDate: It is of Datetime in nature.

* UnitPrice: It is continuous in nature.

* CustomerID: It is categorical and Nominal in nature.

* Country: It is categorical and Nominal in nature.

## **Data Preprocessing**

In this section, I will be treating the null values, dublicated rows and also dropping those orders that have been cancelled by the customers. 

**Note:** Outliers in the data are not removed because we need that data to rank the customers using RFM.

In [31]:
# Checking for duplicate rows.

duplicate_check = retail_df[retail_df.duplicated()]
print(f'The number of duplicate rows in the dataset are {len(duplicate_check)}')

The number of duplicate rows in the dataset are 5268


In [32]:
# Removing all these rows.

retail_df = retail_df[~retail_df.duplicated()]
print('The shape of the dataset after removing duplicates are {0}'.format(retail_df.shape))

The shape of the dataset after removing duplicates are (536641, 8)


In [33]:
# Checking for null values and removing them.

print('The number of null values in each column are as the following: \n')
retail_df.isnull().sum()

The number of null values in each column are as the following: 



InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64

Columns Description and CustomerID have null values. The only way to deal with this is to remove those rows that have null values. Because you can compute CustomerID(unique idenfication number) & Description.

In [34]:
# Dropping all the rows that have null values.

retail_df.dropna(inplace = True)
print('The shape of dataset after dropping null values is {0}'.format(retail_df.shape))

The shape of dataset after dropping null values is (401604, 8)


Even after removing dublicate and null values, we still have a ton of data to play around with.

In [39]:
# Dropping those orders that have been cancelled by the customers.

cancel_df = retail_df[retail_df['InvoiceNo'].str.contains('C',na = False)]
print('The number of cancelled orders by the customers are {0}'.format(len(cancel_df)))

retail_df = retail_df[~retail_df['InvoiceNo'].str.contains('C',na = False)]

The number of cancelled orders by the customers are 8872


Removing cancelled orders because it makes zero sense to consider these observations for RFM analysis.

## **Feature Engineering**

Feature engineering refers to the process of creating new features from exisitng features in the dataset.

In [40]:
# Feature Engineering the Total_Price column.

retail_df['Total_Price'] = retail_df['Quantity'] * retail_df['UnitPrice']
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


The Total_Price can be used to calulate the monetary value of each customer.

In [62]:
# Feature engineering the Last_Purchase column.

assumed_date = dt.datetime(2011,12,10)
retail_df['Last_Purchase'] = (assumed_date - retail_df['InvoiceDate']).dt.days
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Price,Last_Purchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,373
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,373
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373


The Last_Purchase column can be used to calculate the recency value of each customer.